In [1]:
from nsynth_reader import *

In [3]:
data_path = "/workspace/jgusak/Data"
tfrecord_path = "{}/nsynth/nsynth-train.tfrecord".format(data_path)

#####  Read tfrecords without defining a graph
Create generator to iterate through tfrecords 

In [36]:
record_iterator  =  tf.python_io.tf_record_iterator(path=tfrecord_path)

for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    
    
    audio = example.features.feature['audio'].float_list.value
    audio = np.array(audio)

    instrument_family = example.features.feature['instrument_family'].int64_list.value[0]
    instrument_source = example.features.feature['instrument_source'].int64_list.value[0]

    note_str = example.features.feature['note_str'].bytes_list.value[0]
    note = np.frombuffer(note_str, dtype = np.uint8)
    

    pitch = example.features.feature['pitch'].int64_list.value[0]
    qualities = example.features.feature['qualities'].int64_list.value[0]
    velocity = example.features.feature['velocity'].int64_list.value[0]
    
    print(instrument_family, instrument_source,  pitch, qualities, velocity)
    print(note)
    print(audio.shape)
    break

0 2 100 0 100
[ 98  97 115 115  95 115 121 110 116 104 101 116 105  99  95  48  49  56
  45  49  48  48  45  49  48  48]
(64000,)


##### Create dataset

In [4]:
dataset = NSynthDataset(tfrecord_path, is_training=False)

##### Get dataset batch

In [5]:
# get a batch in the following format: tf.Example protobuf parsed from tfrecord
batch = dataset.get_example(batch_size = 10)
print(batch.keys())

batch

dict_keys(['audio', 'instrument_family', 'instrument_source', 'note_str', 'pitch', 'qualities', 'velocity'])


{'audio': <tf.Tensor 'ParseSingleExample/ParseSingleExample:0' shape=(64000,) dtype=float32>,
 'instrument_family': <tf.Tensor 'ParseSingleExample/ParseSingleExample:1' shape=(1,) dtype=int64>,
 'instrument_source': <tf.Tensor 'ParseSingleExample/ParseSingleExample:2' shape=(1,) dtype=int64>,
 'note_str': <tf.Tensor 'ParseSingleExample/ParseSingleExample:3' shape=() dtype=string>,
 'pitch': <tf.Tensor 'ParseSingleExample/ParseSingleExample:4' shape=(1,) dtype=int64>,
 'qualities': <tf.Tensor 'ParseSingleExample/ParseSingleExample:5' shape=(10,) dtype=int64>,
 'velocity': <tf.Tensor 'ParseSingleExample/ParseSingleExample:6' shape=(1,) dtype=int64>}

In [6]:
#get a batch in the following format: {key:tensor} 
batch = dataset.get_wavenet_batch(batch_size = 10)
print(batch.keys())

batch

dict_keys(['pitch', 'wav', 'key'])


{'pitch': <tf.Tensor 'Cast:0' shape=(10,) dtype=int32>,
 'wav': <tf.Tensor 'Reshape_1:0' shape=(10, 64000) dtype=float32>,
 'key': <tf.Tensor 'shuffle_batch:0' shape=(10,) dtype=string>}

##### Define graph to read tfrecords and  iterate through batches

In [13]:
dataset = NSynthDataset(tfrecord_path=tfrecord_path)

LENGTH = 40000
batch = dataset.get_wavenet_batch(batch_size = 50, length = LENGTH) 

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session() as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord = coord)
    
    for i in range(3):
    
        batch_np = sess.run(batch)
        features, labels = batch_np['wav'], batch_np['pitch']

        print(features.shape, labels.shape)
        
    coord.request_stop()
    coord.join(threads)

(50, 40000) (50,)
(50, 40000) (50,)
(50, 40000) (50,)
